In [1]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model


In [2]:
lemmatizer = WordNetLemmatizer()

In [3]:
intents = json.loads(open('intents.json').read())

In [4]:
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [5]:
model = load_model('chatbot_model.model')

In [6]:
def cleanup_sentence(sentence) :
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [7]:
def bagof_words(sentence) :
    sentence_words = cleanup_sentence(sentence)
    bag = [0] * len(words) 
    for w in sentence_words :
        for i, word in enumerate(words) :
            if word == w :
                bag[i] = 1
        return np.array(bag)
        

In [8]:
def predict_class(sentence) :
    bow = bagof_words(sentence) 
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r > ERROR_THRESHOLD]
    
    results.sort(key = lambda x: x[1], reverse = True)
    return_list = []
    for r in results :
        return_list.append({'intent' :classes[r[0]], 'probability' : str(r[1])})
    return return_list

In [9]:
def get_response(intents_list, intents_json) :
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    
    for i in list_of_intents :
        if i['tag'] == tag :
            result = random.choice(i['responses'])
            break
    return result 

In [ ]:
print('GO ! Bot is running!')

while True :
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)

GO ! Bot is running!
hey
Hey :-)
how are you
Hello, thanks for visiting
price
Hey :-)
hey
Hello, thanks for visiting
hi
Hey :-)
whats up?
Hi there, how can I help?
shorty
Hi there, how can I help?
